In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import general_utils as Ugen
import matplotlib.pyplot as plt
import pandas as pd
import math
from pandas.tools.plotting import scatter_matrix
import numpy as np
import glob, os

In [3]:
##GET DATA
#GET FILES
main_dir=os.getcwd()
allFiles = glob.glob(main_dir.split('/Studies')[0]+'/toronto_parking_data'+ "/*.csv")

#LOAD TO DATAFRAME
list_ = []
for file_path in allFiles[-7:]:#allFiles: #START WITH 2014/2015 DATA
    try:
        df = pd.read_csv(file_path,index_col=None,quoting=2,error_bad_lines=False) #quoting=2
        list_.append(df)
        print ('successfully loaded: '+ file_path)
    except Exception as err:
        if 'EOF following escape character' in str(err):
            try:
                df = pd.read_csv(file_path,index_col=None,encoding='utf-16',error_bad_lines=False) #quoting=2
                list_.append(df)
                print ('successfully loaded: '+ file_path)
            except Exception as err:
                print ('NOT loaded: '+ file_path)
                print (str(err))
        else:
            print ('NOT loaded: '+ file_path)
            print (str(err))
tpt_df = pd.concat(list_)
tpt_df.head()

successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_1.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_2.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_3.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2014_4.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2015_1.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2015_2.csv
successfully loaded: /Users/whitesi/Documents/Programming/Python/toronto_parking_data/Parking_Tags_Data_2015_3.csv


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***68436,20140101,134,PARK IN PUBLIC LANE,30,0,R/O,157 SELLERS AVE,NaN,NaN,ON
1,***15940,20140101,192,STAND SIGNED TRANSIT STOP,60,2,NR,288 QUEEN ST W,NaN,NaN,ON
2,***90323,20140101,9,STOP SIGNED HWY PRO TIMES/DAYS,60,3,NR,20 COSBURN AVE,NaN,NaN,ON
3,***90324,20140101,8,STAND SIGN HWY PRO TIMES/DAYS,60,5,NR,30 COSBURN AVE,NaN,NaN,ON
4,***90325,20140101,8,STAND SIGN HWY PRO TIMES/DAYS,60,6,NR,30 COSBURN AVE,NaN,NaN,ON


In [4]:
print (tpt_df.shape)
tpt_df.head()

(4653476, 11)


,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***68436,20140101,134,PARK IN PUBLIC LANE,30,0,R/O,157 SELLERS AVE,NaN,NaN,ON
1,***15940,20140101,192,STAND SIGNED TRANSIT STOP,60,2,NR,288 QUEEN ST W,NaN,NaN,ON
2,***90323,20140101,9,STOP SIGNED HWY PRO TIMES/DAYS,60,3,NR,20 COSBURN AVE,NaN,NaN,ON
3,***90324,20140101,8,STAND SIGN HWY PRO TIMES/DAYS,60,5,NR,30 COSBURN AVE,NaN,NaN,ON
4,***90325,20140101,8,STAND SIGN HWY PRO TIMES/DAYS,60,6,NR,30 COSBURN AVE,NaN,NaN,ON


In [5]:
tpt_df=tpt_df.drop(['tag_number_masked','location3','location4','province'],axis=1)
print (tpt_df.shape)
tpt_df.head()

(4653476, 7)


,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2
0,20140101,134,PARK IN PUBLIC LANE,30,0,R/O,157 SELLERS AVE
1,20140101,192,STAND SIGNED TRANSIT STOP,60,2,NR,288 QUEEN ST W
2,20140101,9,STOP SIGNED HWY PRO TIMES/DAYS,60,3,NR,20 COSBURN AVE
3,20140101,8,STAND SIGN HWY PRO TIMES/DAYS,60,5,NR,30 COSBURN AVE
4,20140101,8,STAND SIGN HWY PRO TIMES/DAYS,60,6,NR,30 COSBURN AVE


In [6]:
tpt_fh_df=tpt_df[tpt_df['infraction_code']==15]
tpt_fh_df.location2.value_counts()

##the following are the top grossing fire hydrants....
##multiply each number by $100 to get total revenue from 2012-2015

393 UNIVERSITY AVE        795
99 ATLANTIC AVE           658
33 ELMHURST AVE           590
5453 YONGE ST             552
361 UNIVERSITY AVE        550
592 MARKHAM ST            441
500 WELLINGTON ST W       422
112 MERTON ST             389
56 THE ESPLANADE          387
5533 YONGE ST             376
5460 YONGE ST             373
17 LILLIAN ST             331
DORIS AVE                 314
5100 YONGE ST             300
6 SPRING GARDEN AVE       287
17 NIAGARA ST             269
21 GLEBE RD E             268
150 KILGOUR RD            258
295 RONCESVALLES AVE      255
123 SUNNYSIDE AVE         252
90 KIPPENDAVIE AVE        243
293 RONCESVALLES AVE      234
387 QUEEN ST W            233
26 WELLINGTON ST E        229
4965 YONGE ST             222
139 WOODWARD AVE          221
49 BELLEVUE AVE           219
5645 YONGE ST             218
4900 YONGE ST             211
43 ELM ST                 200
                         ... 
315 EGLINTON AVE W          1
6 BLUEFIN CRES              1
48 WESTMOR

In [7]:
##LETS FIND THE HIGHEST GROSSING PERIOD
infraction_ids=list(tpt_df.infraction_code.unique())

# tpt_df.groupby(['infraction_code','location2']).size()
#tpt_df.groupby(['infraction_code','location2']).location2.count()

tpt_df_mod=tpt_df.iloc[:500000]
tpt_df_mod=tpt_df_mod.sort_values('location2')

# tpt_df_mod['counts']=tpt_df_mod.groupby(['infraction_code','location2']).location2.transform('count')
# tpt_df_mod

In [8]:
tpt_df_mod.shape
# sorted_df.columns.values

(500000, 7)

In [9]:
#DROP MISSING LOCATIONS
tpt_df_mod=tpt_df_mod[pd.notnull(tpt_df_mod['location2'])]
tpt_df_mod.shape

(499919, 7)

In [10]:
new_df=pd.DataFrame({'count' : tpt_df_mod.groupby( ['infraction_code','location2'] ).size()}).reset_index()
print(new_df.shape)



(142065, 3)


In [11]:
##now calculate total amount...
##create dict of infraction_code:fine_amount
fine_amount_dict={ic:{'fine_amt':tpt_df[tpt_df['infraction_code']==ic].set_fine_amount.median(),
                      'fine_descp':tpt_df[tpt_df['infraction_code']==ic].infraction_description.iloc[0]}    
                          for ic in infraction_ids}


def calc_fine_sum(df):
    return df['count']*fine_amount_dict[df['infraction_code']]['fine_amt']

def get_fine_descrp(df):
    return fine_amount_dict[df['infraction_code']]['fine_descp']

new_df['fine_sum']=new_df.apply(calc_fine_sum,axis=1)
new_df['fine_descrp']=new_df.apply(get_fine_descrp,axis=1)

In [12]:
new_df.sort_values('fine_sum',ascending=False)

,infraction_code,location2,count,fine_sum,fine_descrp
121909,347,939 LAWRENCE AVE E,200,50000,PARK IN A FIRE ROUTE
120987,347,1090 DON MILLS RD,150,37500,PARK IN A FIRE ROUTE
122664,367,18 GRENVILLE ST,83,37350,STND ONSTRT ACCESSIBLE NO PRMT
122702,367,410 COLLEGE ST,83,37350,STND ONSTRT ACCESSIBLE NO PRMT
122484,363,35 BALMUTO ST,79,35550,PARK ONSTRT ACCESSIBLE NO PRMT
24027,5,20 EDWARD ST,877,35080,PARK HWY PROHIBED TIME/DAY
13873,3,2075 BAYVIEW AVE,1043,31290,PARK/LEAVE ON PRIVATE PROPERTY
121630,347,45 OVERLEA BLVD,117,29250,PARK IN A FIRE ROUTE
44179,8,20 EGLINTON AVE W,430,25800,STAND SIGN HWY PRO TIMES/DAYS
12670,3,1265 MILITARY TRL,836,25080,PARK/LEAVE ON PRIVATE PROPERTY


In [13]:
new_df[new_df['location2']=='939 LAWRENCE AVE E']

,infraction_code,location2,count,fine_sum,fine_descrp
17797,3,939 LAWRENCE AVE E,28,840,PARK/LEAVE ON PRIVATE PROPERTY
121909,347,939 LAWRENCE AVE E,200,50000,PARK IN A FIRE ROUTE
122303,355,939 LAWRENCE AVE E,15,6750,PARK IN ACCESSIBLE NO PERMIT
122342,357,939 LAWRENCE AVE E,2,900,PARK ACCESSIBLE NOT DROP OFF


In [14]:
tpt_df.columns

Index(['date_of_infraction', 'infraction_code', 'infraction_description',
       'set_fine_amount', 'time_of_infraction', 'location1', 'location2'],
      dtype='object')

In [15]:
##get unique values of infraction times

tpt_df.time_of_infraction.mean()

1184.5459498081186

In [44]:
import requests

req_url='https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'
api_key = Ugen.ConfigSectionMap('googlemaps')['key']
# api_response = requests.get(req_url.format(address,api_key))
# api_response_dict = api_response.json()


def get_coords(df):
    address=df['location2']+' Toronto, Ontario, Canada'
    try:
        response = requests.get(req_url.format(address,api_key))
        response_dict = response.json()
        coords=response_dict['results'][0]['geometry']['location']
        if response_dict['status'] == 'OK':
            print (response_dict)
            return coords['lat'],coords['lng'],response_dict['results'][0]['formatted_address']
    except Exception as err:
        print (str(err))
        pass
    return 'None','None','None'

##OPTION 2
# from geopy.geocoders import Nominatim
# geolocator = Nominatim()

# def get_coords(df):
#     street=df['location2']
#     try:
#         location = geolocator.geocode(street+" Toronto Ontario Canada")
#         if location.latitude and location.longitude:
#             return {'lon':location.longitude,'lat':location.latitude}
#     except Exception as err:
#         print (str(err))
#         pass
#     return



new2_df=new_df[new_df['infraction_code']==1]
new2_df.shape
new2_df['lat'],new2_df['lng'],new2_df['address']=new2_df.apply(get_coords,axis=1)

{'results': [{'place_id': 'ChIJI_m72kIyK4gRQN_dnD8YpeQ', 'formatted_address': '1 Collinson Blvd, North York, ON M3H 3B7, Canada', 'address_components': [{'long_name': '1', 'short_name': '1', 'types': ['street_number']}, {'long_name': 'Collinson Boulevard', 'short_name': 'Collinson Blvd', 'types': ['route']}, {'long_name': 'Wilson Heights', 'short_name': 'Wilson Heights', 'types': ['neighborhood', 'political']}, {'long_name': 'North York', 'short_name': 'North York', 'types': ['political', 'sublocality', 'sublocality_level_1']}, {'long_name': 'Toronto', 'short_name': 'Toronto', 'types': ['locality', 'political']}, {'long_name': 'Toronto Division', 'short_name': 'Toronto Division', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Ontario', 'short_name': 'ON', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Canada', 'short_name': 'CA', 'types': ['country', 'political']}, {'long_name': 'M3H 3B7', 'short_name': 'M3H 3B7', 'types': ['postal_code']

ValueError: too many values to unpack (expected 3)

In [ ]:
new2_df.head()